# Try the Trained Model
After training, you can use the fine-tuned model for inference. You'll need to load the base Mistral model from the Huggingface Hub and then load the QLoRA adapters from the best-performing checkpoint directory.

In [1]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

/home/adapters_mistral/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# base_model_id = "openchat/openchat_3.5"
base_model_id = "Vikhrmodels/Vikhr-7b-0.1"

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

/home/adapters_mistral/.venv/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:07<00:00,  2.34s/it]


In [4]:
eval_prompt = """Ты квалифицированный бухгалтер, тебе задают профессиональный вопрос. Используй свои знания и как можно точнее отвечай на вопрос. Используй контекст: Учредители и участники организаций могут быть самозанятыми и платить налог на профдоход при оказании услуг и выполнении работ. НПД можно облагать доходы, которые не связаны с работой в качестве учредителя или участника: дивиденды НПД не облагаются. Такой вывод следует из статьи 4 Закона от 27.11.2018 № 422-ФЗ. В рамках НПД учредитель вправе оказывать услуги и выполнять работы в пользу своей организации по ГПД. Например, оказывать юридические услуги по регистрации недвижимости, ремонтировать имущество. Виды деятельности, которые нельзя перевести на НПД, перечислены в пункте 2 статьи 4 и пункте 2 статьи 6 Закона от 27.11.2018 № 422-ФЗ. НПД не вправе платить учредители, у которых с заказчиком действует трудовой договор или у которых трудовой договор расторгнут менее двух лет назад (подп. 8 п. 2 ст. 6 Закона от 27.11.2018 № 422-ФЗ, письмо Минфина от 18.08.2022 № 03-11-11/80767). Получение дивидендов не связано с оказанием услуг и выполнением работ. Это доход, полученный акционером или участником от организации при распределении прибыли (п. 1 ст. 43 НК). Поэтому в отношении дивидендов учредитель не вправе применять НПД. А значит, с дивидендов учредитель обязан уплачивать НДФЛ в общем порядке.  Плательщики налога на профессиональный доход вправе совмещать деятельность на этом спецрежиме с работой по найму. С зарплаты наниматель заплатит НДФЛ и страховые взносы (подп. 8 п. 2 ст. 6 Закона от 27.11.2018 № 422-ФЗ). Налогом на профессиональный доход будут облагаться доходы от переведенных на этот спецрежим видов деятельности (п. 1 ст. 6 Закона от 27.11.2018 № 422-ФЗ, письма ФНС от 28.08.2019 № СД-4-3/17127 и от 25.01.2019 № СД-3-3/458).


### Вопрос:
Добрый день!имеет ли право физ. лицо на основании МЧД подписывать договоры за Ген.директора?если тип полномочий "без ограничений" либо "все полномочия" 


### Ответ:
"""


In [5]:
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.90s/it]


In [7]:
from peft import PeftModel
ft_model = PeftModel.from_pretrained(base_model, "mistral-write-support-bss/checkpoint-400")

/home/adapters_mistral/.venv/lib/python3.10/site-packages/peft/peft_model.py:398: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  adapters_weights = torch.load(


In [8]:
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
ft_model.eval()
with torch.no_grad():
    print(tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=300, pad_token_id=2)[0], skip_special_tokens=True))

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Ты квалифицированный бухгалтер, тебе задают профессиональный вопрос. Используй свои знания и как можно точнее отвечай на вопрос. Используй контекст: Учредители и участники организаций могут быть самозанятыми и платить налог на профдоход при оказании услуг и выполнении работ. НПД можно облагать доходы, которые не связаны с работой в качестве учредителя или участника: дивиденды НПД не облагаются. Такой вывод следует из статьи 4 Закона от 27.11.2018 № 422-ФЗ. В рамках НПД учредитель вправе оказывать услуги и выполнять работы в пользу своей организации по ГПД. Например, оказывать юридические услуги по регистрации недвижимости, ремонтировать имущество. Виды деятельности, которые нельзя перевести на НПД, перечислены в пункте 2 статьи 4 и пункте 2 статьи 6 Закона от 27.11.2018 № 422-ФЗ. НПД не вправе платить учредители, у которых с заказчиком действует трудовой договор или у которых трудовой договор расторгнут менее двух лет назад (подп. 8 п. 2 ст. 6 Закона от 27.11.2018 № 422-ФЗ, письмо Минф